In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy
import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu
import statsmodels.api as sm

import Models.Soybean_USA_Yield_GA as sy

import APIs.Bloomberg as ba
from sklearn.model_selection import TimeSeriesSplit

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [ ]:
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope) 

multi_ww_dt_s=dt(2022,5,1)
multi_ww_dt_e=dt(2022,9,1)

multi_ww_freq_start='1D'
multi_ww_freq_end='1D'

multi_ww_ref_year_s=dt(2022,1,1)
    
raw_data['multi_ww_df']=um.generate_weather_windows_df(raw_data['w_w_df_all']['hist'], date_start=multi_ww_dt_s, date_end=multi_ww_dt_e, ref_year_start=multi_ww_ref_year_s, freq_start=multi_ww_freq_start, freq_end=multi_ww_freq_end)


In [ ]:
model_df = pd.concat([raw_data['yield'],raw_data['multi_ww_df']], sort=True, axis=1, join='inner')
model_df=model_df.dropna() # Needed because (maybe) the current year has some windows that have not started yet

# Train Test Splits 
min_train_size= min(10,len(model_df)-3)
folds_expanding = TimeSeriesSplit(n_splits=len(model_df)-min_train_size, max_train_size=0, test_size=1)
folds = []
folds = folds + list(folds_expanding.split(model_df))

In [ ]:
y_col  ='Yield'
y_df = model_df[[y_col]]

In [ ]:
X_df = deepcopy(model_df)

cols=list(m.params.index)
cols.remove('const')
X_df=X_df[cols]


cv_score = um.stats_model_cross_validate(X_df, y_df, folds)

In [ ]:
np.mean(cv_score['cv_p'])

In [ ]:
X_df.head()

In [ ]:
np.mean(raw_data['w_w_df_all']['hist'][['USA_TempMax']].loc['1985-08-20':'1985-08-27'])

In [ ]:
model_df.head()

In [ ]:
m = um.pick_model('GA_soy',72)
ww = um.from_cols_to_var_windows(m.params.index)

In [ ]:
model_df = um.extract_yearly_ww_variables(w_df = raw_data['w_w_df_all']['hist'],var_windows= ww)
model_df['year']=model_df.index

In [ ]:
np.mean(raw_data['w_w_df_all']['hist'][['USA_TempMax']].loc['1985-08-21':'1985-08-28'])

In [ ]:
y_df = raw_data['yield'][raw_data['yield'].index<2022][['Yield']]

In [ ]:
X_df=model_df

folds = um.folds_expanding(X_df, min_train_size=10)
# um.print_folds(folds = folds, years=train_df.index.values)

# Cross-Validation calculation
cv_score = um.stats_model_cross_validate(X_df, y_df, folds)

In [ ]:
m.params

In [ ]:
np.mean(cv_score['cv_p'])

In [ ]:
f='GA_soy'
dm_best = {}
dm_best[f] = uu.deserialize(f)
r=dm_best[f]

r.keys()

In [ ]:
np.mean(r['cv_p'][72])